In [1]:
import os
import linecache

import random
import math
import copy
import sys
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm
from tqdm import trange

from itertools import chain, cycle, combinations

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, train_test_split
from sklearn.preprocessing import label_binarize, StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder
from joblib import parallel_backend

## Step 1: get 1x1 dataset

In [2]:
WORKING_PATH = r"C:\Users\zhao-\Jiaheng\BSA protein\data"
df = pd.read_csv(WORKING_PATH + r'\BSA-thiol_dataset_corrected.csv')
df

C:\Users\zhao-\AppData\Local\Temp\ipykernel_72072\168126270.py:2: DtypeWarning: Columns (1401,1402,1403) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(WORKING_PATH + r'\BSA-thiol_dataset_corrected.csv')


,401,402,403,404,405,406,407,408,409,410,...,1795,1796,1797,1798,1799,label,conc,batch,original_filename,spectrum_id
0,0.070472,0.019269,0.049263,0.061706,0.092834,0.047193,0.043878,0.064576,0.047192,0.045215,...,0.576888,0.530230,0.537368,0.515348,0.554981,Cysteamine-BSA,0.0,NaN,NaN,NaN
1,0.052520,0.015020,0.073222,0.129306,0.074067,0.051379,0.087747,0.126177,0.070515,0.067214,...,0.721957,0.759064,0.741285,0.725900,0.732966,Cysteamine-BSA,0.0,NaN,NaN,NaN
2,0.058144,0.061932,0.067105,0.079980,0.103823,0.079320,0.041960,0.079235,0.040840,0.004985,...,0.682711,0.688279,0.659778,0.666903,0.655693,Cysteamine-BSA,0.0,NaN,NaN,NaN
3,0.041245,0.078781,0.064747,0.100347,0.141697,0.052391,0.059124,0.002798,0.032093,0.088595,...,0.550651,0.579228,0.591954,0.533540,0.564195,Cysteamine-BSA,0.0,NaN,NaN,NaN
4,0.044159,0.025640,0.080267,0.119340,0.137719,0.097517,0.151911,0.150072,0.083216,0.125389,...,0.646457,0.653475,0.597139,0.597022,0.600918,Cysteamine-BSA,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,0.295345,0.458984,0.336862,0.368042,0.370117,0.278304,0.328628,0.192342,0.201891,0.291367,...,0.860684,1.088699,0.905192,0.788690,0.743406,Bare-BSA,0.0,blank,blank.csv-Corrected-Normalized_by_Mean.csv,blank_56
1490,0.039019,0.012957,0.080528,0.208564,0.243892,0.091088,0.076445,0.177045,0.339447,0.376123,...,1.068873,0.373567,1.000725,0.902617,0.594679,Bare-BSA,0.0,blank,blank.csv-Corrected-Normalized_by_Mean.csv,blank_57
1491,0.421393,0.543113,0.370042,0.364926,0.254257,0.251780,0.498346,0.613888,0.519034,0.187689,...,0.971034,1.093031,0.890048,0.900299,0.867759,Bare-BSA,0.0,blank,blank.csv-Corrected-Normalized_by_Mean.csv,blank_58
1492,0.297975,0.191096,0.285573,0.328852,0.507006,0.384780,0.196296,0.159697,0.270925,0.279471,...,0.881148,1.126206,1.056573,0.852025,1.067942,Bare-BSA,0.0,blank,blank.csv-Corrected-Normalized_by_Mean.csv,blank_59


#### Read the whole dataset.
We assume the dataset is **one csv file** containing all the spectra.

This csv file must satisfy:
1. Every row is a spectrum.
2. Every column except for the last two represents a wavenumber.
3. The header of the last two columns are ```Label``` (virus type) and ```Conc``` (concentration).

If you have multiple txt files or csv files, please use corresponding code **(to be done)** to process it first.

Please remember to change the path in ```r' '```, and the path should begin with ```\```.

In [ ]:
wavenumbers = np.array([int(x) for x in df.columns.tolist()[:-2]])
wavenumbers

In [ ]:
labels = np.unique(df['label'])
labels

### Dataset split

In [5]:
# set the number of total iterations (groups)
iteration_number = 10
START = 1 # Included

# Adjust the proportions of test set (default = 1/3).
proportion_test = 1/3

In [7]:
for i in range(START, iteration_number+START):
    # split (train & val) and test.
    train = df.groupby(['label','conc'], group_keys=False).apply(lambda x: x.sample(frac=1-proportion_test))
    test = df.loc[df.index.difference(train.index)]
    # train.shape, test.shape
    
    # create subfolder if not exists
    subfolder_path = os.path.join(WORKING_PATH, f"BSAML101_test_train_dataset", f"train_test_set{i}")
    os.makedirs(subfolder_path, exist_ok=True)

    # save train & test to csv
    train.to_csv(os.path.join(subfolder_path, f"train_dataset{i}.csv"), index=False)
    test.to_csv(os.path.join(subfolder_path, f"test_dataset{i}.csv"), index=False)

### Save the train and test to 4 files for 4 substrates

In [8]:
for i in range(START, iteration_number+START):
    # train
    original_file_path = f"{WORKING_PATH}\\BSAML101_test_train_dataset\\train_test_set{i}\\train_dataset{i}.csv"

    df = pd.read_csv(original_file_path)

    # Get the name of the second to last column
    label_column = df.columns[-2]

    # Determine the path of the original file
    directory = os.path.dirname(original_file_path)

    # Create a new folder named 'split' in the same directory as the original file
    split_folder = os.path.join(directory, 'train_split')
    os.makedirs(split_folder, exist_ok=True)

    # Group the data by the label column
    grouped = df.groupby(label_column)

    # Iterate over each group and save to a separate CSV file in the 'split' folder
    for label, group in grouped:
        # Create a valid file name by replacing invalid characters
        label_name = str(label).replace('/', '_').replace('\\', '_')

        # Define the new file path
        new_file_path = os.path.join(split_folder, f'{label_name}.csv')

        # Save the group to a CSV file
        group.to_csv(new_file_path, index=False)

    print(f"The {i}-th TRAINING CSV files have been created for each label and saved in the 'train_split' directory.")
    
    
    # test
    original_file_path = f"{WORKING_PATH}\\BSAML101_test_train_dataset\\train_test_set{i}\\test_dataset{i}.csv"
    df = pd.read_csv(original_file_path)

    # Get the name of the second to last column
    label_column = df.columns[-2]

    # Determine the path of the original file
    directory = os.path.dirname(original_file_path)

    # Create a new folder named 'split' in the same directory as the original file
    split_folder = os.path.join(directory, 'test_split')
    os.makedirs(split_folder, exist_ok=True)

    # Group the data by the label column
    grouped = df.groupby(label_column)

    # Iterate over each group and save to a separate CSV file in the 'split' folder
    for label, group in grouped:
        # Create a valid file name by replacing invalid characters
        label_name = str(label).replace('/', '_').replace('\\', '_')

        # Define the new file path
        new_file_path = os.path.join(split_folder, f'{label_name}.csv')

        # Save the group to a CSV file
        group.to_csv(new_file_path, index=False)

    print(f"The {i}-th TESTING CSV files have been created for each label and saved in the 'test_split' directory.")

The 1-th TRAINING CSV files have been created for each label and saved in the 'train_split' directory.
The 1-th TESTING CSV files have been created for each label and saved in the 'test_split' directory.
The 2-th TRAINING CSV files have been created for each label and saved in the 'train_split' directory.
The 2-th TESTING CSV files have been created for each label and saved in the 'test_split' directory.
The 3-th TRAINING CSV files have been created for each label and saved in the 'train_split' directory.
The 3-th TESTING CSV files have been created for each label and saved in the 'test_split' directory.
The 4-th TRAINING CSV files have been created for each label and saved in the 'train_split' directory.
The 4-th TESTING CSV files have been created for each label and saved in the 'test_split' directory.
The 5-th TRAINING CSV files have been created for each label and saved in the 'train_split' directory.
The 5-th TESTING CSV files have been created for each label and saved in the 'tes

## Step 2: Generate super spectra

In [9]:
# draw from the spectra with replacement, but without duplicate
# note: without replacement = every spectra will only appear at most once, this will result in at most 40 spectra for training, 20 spectra for testing
# with replacement = spectra may appear more than once. If without duplicate, this will result in at most 40*40 spectra for training, 20*20 spectra for testing
def draw_with_replacement(*dataframes, conc_levels, num_super_spectra):
    super_spectra_list = []

    # Iterate over each concentration level
    for conc in conc_levels:
        generated_spectra = set()  # Set to keep track of generated spectra to avoid duplicates
        count = 0

        while count < num_super_spectra:
            current_spectra = []
            all_spectra_present = True

            for df in dataframes:
                # Filter the dataframe for the current concentration
                df_conc = df[df['conc'] == conc].reset_index(drop=True)

                # Check if any dataframe does not contain the concentration
                if df_conc.empty:
                    all_spectra_present = False
                    print(f"No spectra found for concentration {conc} in one of the dataframes.")
                    break  # Break out of the inner loop

                # Draw a random index with replacement
                chosen_index = np.random.choice(df_conc.index, 1)[0]

                # Select the spectrum without the last two columns ('Label', 'Conc')
                spectrum = df_conc.iloc[chosen_index, :-2].values
                current_spectra.extend(spectrum)

            # Convert list to tuple for immutability and set operations
            current_spectra_tuple = tuple(current_spectra)

            # If a spectrum was found in each dataframe for the current concentration
            if all_spectra_present:
                if current_spectra_tuple not in generated_spectra:
                    generated_spectra.add(current_spectra_tuple)
                    # Assume all dataframes have the same Label for the same concentration
                    label = df_conc.iloc[0]['label']
                    super_spectra_list.append(list(current_spectra_tuple) + [label, conc])
                    count += 1

    # Determine the column names for the super spectrum dataframe
    col_names = []
    for i, df in enumerate(dataframes, 1):
        col_names.extend([f'wavenumber_{i}_{col}' for col in range(len(df.columns) - 2)])
    col_names.extend(['label', 'conc'])

    # Combine all the super spectra into a single DataFrame
    super_spectra_df = pd.DataFrame(super_spectra_list, columns=col_names)
    
    return super_spectra_df

### Output all possible combinations

In [10]:
def save_all_combinations(START, iteration_number, WORKING_PATH, conc_levels):
    for i in range(START, iteration_number + START):
        # Initialize dictionaries to store training and testing DataFrames
        train_dfs = {}
        test_dfs = {}

        # List of modification types
        modifications = ["Bare-BSA", "Cysteamine-BSA", "Cysteine-BSA", "MCH-BSA"]
        
        # Loop through each modification and load its CSV into a DataFrame for both training and testing
        for modification in modifications:
            # Process train DataFrames
            train_path = os.path.join(WORKING_PATH, f"BSAML101_test_train_dataset\\train_test_set{i}\\train_split", f"{modification}.csv")
            train_dfs[modification] = pd.read_csv(train_path)
            
            # Process test DataFrames
            test_path = os.path.join(WORKING_PATH, f"BSAML101_test_train_dataset\\train_test_set{i}\\test_split", f"{modification}.csv")
            test_dfs[modification] = pd.read_csv(test_path)

        # Process and save combinations of 2, 3, and 4 modifications
        for num_mods in [2, 3, 4]:
            for phase, dfs in [('train', train_dfs), ('test', test_dfs)]:
                # Iterate over combinations of modifications
                for combo in combinations(dfs.items(), num_mods):
                    # Concatenate the DataFrames along the columns
                    dataframes = [df.iloc[:, :-2] for mod, df in combo[:-1]] + [dfs[combo[-1][0]]]  # Include all columns for the last DataFrame
                    combined_df = pd.concat(dataframes, axis=1)
                    
                    # Clean and sort modification names for the filename
                    mods_clean = sorted([mod.replace('-BSA', '') for mod, _ in combo])
                    mods_filename = '_'.join(mods_clean)
                    
                    # Define file path for saving the CSV
                    filename = os.path.join(WORKING_PATH, f"Superspectra20250705/{num_mods}x{num_mods}_{i}", f"{mods_filename}_{phase}{i}.csv")
                    os.makedirs(os.path.dirname(filename), exist_ok=True)
                    combined_df.to_csv(filename, index=False)
                    print(f"Saved: {filename}")


In [11]:
# Example usage
START = 1
iteration_number = 10
conc_levels = [0, 0.05, 0.001, 0.00002, 4e-7, 8e-9]

save_all_combinations(START, iteration_number, WORKING_PATH, conc_levels)

Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Bare_Cysteamine_train1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Bare_Cysteine_train1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Bare_MCH_train1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Cysteamine_Cysteine_train1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Cysteamine_MCH_train1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Cysteine_MCH_train1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Bare_Cysteamine_test1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Bare_Cysteine_test1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Bare_MCH_test1.csv
Saved: C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107/2x2_1\Cysteamine_Cysteine_test1.csv
Saved: C

## Step 3: regression models

### 1x1 single spectra

In [12]:
# set the number of total iterations (groups)
iteration_number = 5
START = 1 # Included

In [13]:
for i in range(START, START+iteration_number):
    modification_combination_list = ['Bare-BSA', 'Cysteamine-BSA', 'Cysteine-BSA', 'MCH-BSA']
    
    for modification in modification_combination_list:
        # Step 1: reformat the train and test sets
        # train
        file_path = f"{WORKING_PATH}\\BSAML101_test_train_dataset\\train_test_set{i}\\train_dataset{i}.csv"

        # Read the CSV file into a DataFrame
        train = pd.read_csv(file_path)

#         # Display the first few rows of the data to verify
#         print(train.head())

        # Select training data with a specific label
        selected_label = train['label'].iloc[0] # always choose the first label. This label has no use, but just to ensure that the code can run.
        selected_data_train = train[train['label'] == selected_label] # split training data
        # selected_data_train

        # Get the indices of the selected data
        selected_indices_train = selected_data_train.index
        # selected_indices_train

        # Get new X_train and y_train_reg
        X_train_split = selected_data_train.values[:,:-2]
        y_train_reg_split = np.log10(selected_data_train.values[:,-1].astype(float)+1e-20) # we only need the regression part.

        # test
        tfile_path = f"{WORKING_PATH}\\BSAML101_test_train_dataset\\train_test_set{i}\\test_dataset{i}.csv"

        # Read the CSV file into a DataFrame
        test = pd.read_csv(tfile_path)

#         # Display the first few rows of the data to verify
#         print(test.head())

        # Similarly, select testing data with the specific label
        selected_data_test = test[test['label'] == selected_label] # split testing data
        # selected_data_test

        # Get the indices of the selected data
        selected_indices_test = selected_data_test.index
        # selected_indices_test

        # Get new X_test and y_test_reg
        X_test_split = selected_data_test.values[:,:-2]
        y_test_reg_split = np.log10(selected_data_test.values[:,-1].astype(float)+1e-20) # we only need the regression part.
     
    
        # Step 2: SVR calculation
        # Create a svr
        svr = svm.SVR(kernel = 'rbf', C = 1000, epsilon = 0.1)
        # sklearn.svr only accept label vectors as input, not one-hot vectors.
        y_train_reg_split_label = y_train_reg_split
        with parallel_backend('threading', n_jobs=-1):
            # Fit the SVR model
            svr.fit(X_train_split, y_train_reg_split_label)
        svr_pred = svr.predict(X_test_split)
        
        # Comparing results
        y_test_reg_split_label = y_test_reg_split

        # Saving performances and pred vs real curve
        MAE = mean_absolute_error(y_test_reg_split_label, svr_pred)
        RMSE = np.sqrt(mean_squared_error(y_test_reg_split_label, svr_pred))
        R_squared = r2_score(y_test_reg_split_label, svr_pred)

        print("MAE:", MAE)
        print("RMSE:", RMSE)
        print("R^2:", R_squared)
        
        # New directory where you want to save the file
        save_directory = f"{WORKING_PATH}\\Superspectra20250705\\Results\\1x1_{i}_FigureData"
        os.makedirs(save_directory, exist_ok=True)
#         # Extract the base filename from the original file_path
#         base_filename = os.path.basename(file_path)

        # Create a DataFrame with these metrics
        performance_metrics_df = pd.DataFrame({
            'MAE': [MAE],
            'RMSE': [RMSE],
            'R^2': [R_squared]
        })
        
        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_performance_svr{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        performance_metrics_df.to_csv(save_file_path, index=False)
        
        # Actual and predicted values
#         plt.figure(figsize=(6, 6))
#         plt.scatter(y_test_reg_split, svr_pred, edgecolors="grey", linewidths=0.2, alpha = 0.3)
#         plt.axline([plt.xlim()[0], plt.xlim()[0]], [plt.xlim()[1], plt.xlim()[1]], linestyle='--', color="grey")
#         plt.xlabel("Actual log10(C)")
#         plt.ylabel("Predicted log10(C)")
#         plt.title(modification)
        
        plot_data_df = pd.DataFrame({
            'Actual_log10(C)': y_test_reg_split,
            'Predicted_log10(C)': svr_pred
        })

        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_svr{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        plot_data_df.to_csv(save_file_path, index=False)

        print(f"SVR plot data saved to {save_file_path}")

        
        # Step 3: RF calculation
        # Create an rf
        rf = RandomForestRegressor(n_estimators=100, criterion="squared_error", max_depth=None, n_jobs=-1, verbose=0)
        rf.fit(X_train_split, y_train_reg_split_label)
        rf_pred = rf.predict(X_test_split)

        # Saving performances and pred vs real curve
        MAE = mean_absolute_error(y_test_reg_split_label, rf_pred)
        RMSE = np.sqrt(mean_squared_error(y_test_reg_split_label, rf_pred))
        R_squared = r2_score(y_test_reg_split_label, rf_pred)

        print("MAE:", MAE)
        print("RMSE:", RMSE)
        print("R^2:", R_squared)
       
        # Create a DataFrame with these metrics
        performance_metrics_df = pd.DataFrame({
            'MAE': [MAE],
            'RMSE': [RMSE],
            'R^2': [R_squared]
        })
        
        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_performance_rf{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        performance_metrics_df.to_csv(save_file_path, index=False)
        
        # Actual and predicted values
#         plt.figure(figsize=(6, 6))
#         plt.scatter(y_test_reg_split, rf_pred, edgecolors="grey", linewidths=0.2, alpha = 0.3)
#         plt.axline([plt.xlim()[0], plt.xlim()[0]], [plt.xlim()[1], plt.xlim()[1]], linestyle='--', color="grey")
#         plt.xlabel("Actual log10(C)")
#         plt.ylabel("Predicted log10(C)")
#         plt.title(modification)

        plot_data_df = pd.DataFrame({
            'Actual_log10(C)': y_test_reg_split,
            'Predicted_log10(C)': rf_pred
        })

        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_rf{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        plot_data_df.to_csv(save_file_path, index=False)

        print(f"RF plot data saved to {save_file_path}")


MAE: 1.2609676979141937
RMSE: 1.942515770299907
R^2: 0.8536016429220932
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\1x1_1_FigureData\Bare-BSA_svr1.csv
MAE: 1.4281223364611553
RMSE: 2.152962050300328
R^2: 0.8201626701292901
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\1x1_1_FigureData\Bare-BSA_rf1.csv
MAE: 1.2609676979141937
RMSE: 1.942515770299907
R^2: 0.8536016429220932
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\1x1_1_FigureData\Cysteamine-BSA_svr1.csv
MAE: 1.3896189031422765
RMSE: 2.0745481293218053
R^2: 0.833023971920293
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\1x1_1_FigureData\Cysteamine-BSA_rf1.csv
MAE: 1.2609676979141937
RMSE: 1.942515770299907
R^2: 0.8536016429220932
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\1x1_1_FigureData\Cysteine-BSA_sv

### 2x2 superspectra

In [14]:
# set the number of total iterations (groups)
iteration_number = 10
START = 1 # Included

In [15]:
from sklearn.impute import SimpleImputer

def preprocess_data(df):
    # Check if any NaNs are in the DataFrame
    if df.isna().any().any():
        print("NaN values found in the DataFrame. Imputing with median values...")
        # Impute NaN values using median, which is robust to outliers
        imputer = SimpleImputer(strategy='median')
        df.iloc[:, :-2] = imputer.fit_transform(df.iloc[:, :-2])  # Assume last two columns are 'label' and 'conc'
    return df

for i in range(START, START+iteration_number):
    modification_combination_list = ['Bare_Cysteamine', 'Bare_Cysteine', 'Bare_MCH', 'Cysteamine_Cysteine', 'Cysteamine_MCH', 'Cysteine_MCH']
    
    for modification in modification_combination_list:
        # Step 1: reformat the train and test sets
        # train
        file_path = f"{WORKING_PATH}\\Superspectra20250705\\2x2_{i}\\{modification}_train{i}.csv"

        # Read the CSV file into a DataFrame
        train = pd.read_csv(file_path)
        train = preprocess_data(train)

#         # Display the first few rows of the data to verify
#         print(train.head())

        # Select training data with a specific label
        selected_label = train['label'].iloc[0] # always choose the first label. This label has no use, but just to ensure that the code can run.
        selected_data_train = train[train['label'] == selected_label] # split training data
        # selected_data_train

        # Get the indices of the selected data
        selected_indices_train = selected_data_train.index
        # selected_indices_train

        # Get new X_train and y_train_reg
        X_train_split = selected_data_train.values[:,:-2]
        y_train_reg_split = np.log10(selected_data_train.values[:,-1].astype(float)+1e-20) # we only need the regression part.

        # test
        tfile_path = f"{WORKING_PATH}\\Superspectra20250705\\2x2_{i}\\{modification}_test{i}.csv"  # Replace with the path to your CSV file

        # Read the CSV file into a DataFrame
        test = pd.read_csv(tfile_path)
        test = preprocess_data(test)

#         # Display the first few rows of the data to verify
#         print(test.head())

        # Similarly, select testing data with the specific label
        selected_data_test = test[test['label'] == selected_label] # split testing data
        # selected_data_test

        # Get the indices of the selected data
        selected_indices_test = selected_data_test.index
        # selected_indices_test

        # Get new X_test and y_test_reg
        X_test_split = selected_data_test.values[:,:-2]
        y_test_reg_split = np.log10(selected_data_test.values[:,-1].astype(float)+1e-20) # we only need the regression part.
     
    
        # Step 2: SVR calculation
        # Create a svr
        svr = svm.SVR(kernel = 'rbf', C = 1000, epsilon = 0.1)
        # sklearn.svr only accept label vectors as input, not one-hot vectors.
        y_train_reg_split_label = y_train_reg_split
        with parallel_backend('threading', n_jobs=-1):
            # Fit the SVR model
            svr.fit(X_train_split, y_train_reg_split_label)
        svr_pred = svr.predict(X_test_split)
        
        # Comparing results
        y_test_reg_split_label = y_test_reg_split

        # Saving performances and pred vs real curve
        MAE = mean_absolute_error(y_test_reg_split_label, svr_pred)
        RMSE = np.sqrt(mean_squared_error(y_test_reg_split_label, svr_pred))
        R_squared = r2_score(y_test_reg_split_label, svr_pred)

        print("MAE:", MAE)
        print("RMSE:", RMSE)
        print("R^2:", R_squared)
        
        # New directory where you want to save the file
        save_directory = f"{WORKING_PATH}\\Superspectra20250705\\Results\\2x2_{i}_FigureData"
        os.makedirs(save_directory, exist_ok=True)
        
#         # Extract the base filename from the original file_path
#         base_filename = os.path.basename(file_path)

        # Create a DataFrame with these metrics
        performance_metrics_df = pd.DataFrame({
            'MAE': [MAE],
            'RMSE': [RMSE],
            'R^2': [R_squared]
        })
        
        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_performance_svr{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        performance_metrics_df.to_csv(save_file_path, index=False) 
        
        # Actual and predicted values
#         plt.figure(figsize=(6, 6))
#         plt.scatter(y_test_reg_split, svr_pred, edgecolors="grey", linewidths=0.2, alpha = 0.3)
#         plt.axline([plt.xlim()[0], plt.xlim()[0]], [plt.xlim()[1], plt.xlim()[1]], linestyle='--', color="grey")
#         plt.xlabel("Actual log10(C)")
#         plt.ylabel("Predicted log10(C)")
#         plt.title(modification)
        
        plot_data_df = pd.DataFrame({
            'Actual_log10(C)': y_test_reg_split,
            'Predicted_log10(C)': svr_pred
        })

        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_svr{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        plot_data_df.to_csv(save_file_path, index=False)

        print(f"SVR plot data saved to {save_file_path}")

        
        # Step 3: RF calculation
        # Create an rf
        rf = RandomForestRegressor(n_estimators=100, criterion="squared_error", max_depth=None, n_jobs=-1, verbose=0)
        rf.fit(X_train_split, y_train_reg_split_label)
        rf_pred = rf.predict(X_test_split)

        # Saving performances and pred vs real curve
        MAE = mean_absolute_error(y_test_reg_split_label, rf_pred)
        RMSE = np.sqrt(mean_squared_error(y_test_reg_split_label, rf_pred))
        R_squared = r2_score(y_test_reg_split_label, rf_pred)

        print("MAE:", MAE)
        print("RMSE:", RMSE)
        print("R^2:", R_squared)
        
        # Create a DataFrame with these metrics
        performance_metrics_df = pd.DataFrame({
            'MAE': [MAE],
            'RMSE': [RMSE],
            'R^2': [R_squared]
        })
        
        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_performance_rf{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        performance_metrics_df.to_csv(save_file_path, index=False)
        
        # Actual and predicted values
#         plt.figure(figsize=(6, 6))
#         plt.scatter(y_test_reg_split, rf_pred, edgecolors="grey", linewidths=0.2, alpha = 0.3)
#         plt.axline([plt.xlim()[0], plt.xlim()[0]], [plt.xlim()[1], plt.xlim()[1]], linestyle='--', color="grey")
#         plt.xlabel("Actual log10(C)")
#         plt.ylabel("Predicted log10(C)")
#         plt.title(modification)

        plot_data_df = pd.DataFrame({
            'Actual_log10(C)': y_test_reg_split,
            'Predicted_log10(C)': rf_pred
        })

        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_rf{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        plot_data_df.to_csv(save_file_path, index=False)

        print(f"RF plot data saved to {save_file_path}")


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\2281128983.py:21: DtypeWarning: Columns (2798) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


NaN values found in the DataFrame. Imputing with median values...
NaN values found in the DataFrame. Imputing with median values...
MAE: 3.2091488778761783
RMSE: 4.698540119177747
R^2: 0.1625249136157637
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_1_FigureData\Bare_Cysteamine_svr1.csv
MAE: 2.1136249553807556
RMSE: 3.8974566299995854
R^2: 0.4237531032253342
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_1_FigureData\Bare_Cysteamine_rf1.csv


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\2281128983.py:21: DtypeWarning: Columns (2798) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


NaN values found in the DataFrame. Imputing with median values...
NaN values found in the DataFrame. Imputing with median values...
MAE: 2.131583159431404
RMSE: 3.4404062800580495
R^2: 0.5719569547010224
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_1_FigureData\Bare_Cysteine_svr1.csv
MAE: 0.26346159624534776
RMSE: 0.48621122015580137
R^2: 0.9914509484647269
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_1_FigureData\Bare_Cysteine_rf1.csv


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\2281128983.py:21: DtypeWarning: Columns (2798) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


...

NaN values found in the DataFrame. Imputing with median values...
NaN values found in the DataFrame. Imputing with median values...
MAE: 3.158129762546185
RMSE: 4.677286233038254
R^2: 0.17008442842632931
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_10_FigureData\Bare_Cysteamine_svr10.csv
MAE: 2.0135853541404503
RMSE: 2.6042302617723094
R^2: 0.7427207389459461
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_10_FigureData\Bare_Cysteamine_rf10.csv


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\2281128983.py:21: DtypeWarning: Columns (2798) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


NaN values found in the DataFrame. Imputing with median values...
NaN values found in the DataFrame. Imputing with median values...
MAE: 2.7293801132268336
RMSE: 3.9872261033803076
R^2: 0.4250769651619757
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_10_FigureData\Bare_Cysteine_svr10.csv
MAE: 0.2879280888368856
RMSE: 0.5096620410123128
R^2: 0.9906063892541225
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_10_FigureData\Bare_Cysteine_rf10.csv


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\2281128983.py:21: DtypeWarning: Columns (2798) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


NaN values found in the DataFrame. Imputing with median values...
NaN values found in the DataFrame. Imputing with median values...
MAE: 2.2946860868584924
RMSE: 3.662563862347412
R^2: 0.4764885448957631
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_10_FigureData\Bare_MCH_svr10.csv
MAE: 1.309697592494323
RMSE: 3.284484146063358
R^2: 0.5789922244696688
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_10_FigureData\Bare_MCH_rf10.csv
NaN values found in the DataFrame. Imputing with median values...
NaN values found in the DataFrame. Imputing with median values...
MAE: 0.5895248102963022
RMSE: 0.7923815631198247
R^2: 0.9772942053607913
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\2x2_10_FigureData\Cysteamine_Cysteine_svr10.csv
MAE: 0.23202981771249162
RMSE: 0.42790554000377334
R^2: 0.9933783869187732
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA 

### 3x3 superspectra

In [16]:
# set the number of total iterations (groups)
iteration_number = 10
START = 1 # Included

In [17]:
for i in range(START, START+iteration_number):
    modification_combination_list = ['Bare_Cysteamine_Cysteine', 'Bare_Cysteamine_MCH', 'Bare_Cysteine_MCH', 'Cysteamine_Cysteine_MCH']
    
    for modification in modification_combination_list:
        # Step 1: reformat the train and test sets
        # train
        file_path = f"{WORKING_PATH}\\Superspectra20250705\\3x3_{i}\\{modification}_train{i}.csv"

        # Read the CSV file into a DataFrame
        train = pd.read_csv(file_path)
        train = preprocess_data(train)

#         # Display the first few rows of the data to verify
#         print(train.head())

        # Select training data with a specific label
        selected_label = train['label'].iloc[0] # always choose the first label. This label has no use, but just to ensure that the code can run.
        selected_data_train = train[train['label'] == selected_label] # split training data
        # selected_data_train

        # Get the indices of the selected data
        selected_indices_train = selected_data_train.index
        # selected_indices_train

        # Get new X_train and y_train_reg
        X_train_split = selected_data_train.values[:,:-2]
        y_train_reg_split = np.log10(selected_data_train.values[:,-1].astype(float)+1e-20) # we only need the regression part.

        # test
        tfile_path = f"{WORKING_PATH}\\Superspectra20250705\\3x3_{i}\\{modification}_test{i}.csv"  # Replace with the path to your CSV file

        # Read the CSV file into a DataFrame
        test = pd.read_csv(tfile_path)
        test = preprocess_data(test)

#         # Display the first few rows of the data to verify
#         print(test.head())

        # Similarly, select testing data with the specific label
        selected_data_test = test[test['label'] == selected_label] # split testing data
        # selected_data_test

        # Get the indices of the selected data
        selected_indices_test = selected_data_test.index
        # selected_indices_test

        # Get new X_test and y_test_reg
        X_test_split = selected_data_test.values[:,:-2]
        y_test_reg_split = np.log10(selected_data_test.values[:,-1].astype(float)+1e-20) # we only need the regression part.
     
    
        # Step 2: SVR calculation
        # Create a svr
        svr = svm.SVR(kernel = 'rbf', C = 1000, epsilon = 0.1)
        # sklearn.svr only accept label vectors as input, not one-hot vectors.
        y_train_reg_split_label = y_train_reg_split
        with parallel_backend('threading', n_jobs=-1):
            # Fit the SVR model
            svr.fit(X_train_split, y_train_reg_split_label)
        svr_pred = svr.predict(X_test_split)
        
        # Comparing results
        y_test_reg_split_label = y_test_reg_split

        # Saving performances and pred vs real curve
        MAE = mean_absolute_error(y_test_reg_split_label, svr_pred)
        RMSE = np.sqrt(mean_squared_error(y_test_reg_split_label, svr_pred))
        R_squared = r2_score(y_test_reg_split_label, svr_pred)

        print("MAE:", MAE)
        print("RMSE:", RMSE)
        print("R^2:", R_squared)
        
        # New directory where you want to save the file
        save_directory = f"{WORKING_PATH}\\Superspectra20250705\\Results\\3x3_{i}_FigureData"
        os.makedirs(save_directory, exist_ok=True)
        
#         # Extract the base filename from the original file_path
#         base_filename = os.path.basename(file_path)

        # Create a DataFrame with these metrics
        performance_metrics_df = pd.DataFrame({
            'MAE': [MAE],
            'RMSE': [RMSE],
            'R^2': [R_squared]
        })
        
        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_performance_svr{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        performance_metrics_df.to_csv(save_file_path, index=False) 
        
        # Actual and predicted values
#         plt.figure(figsize=(6, 6))
#         plt.scatter(y_test_reg_split, svr_pred, edgecolors="grey", linewidths=0.2, alpha = 0.3)
#         plt.axline([plt.xlim()[0], plt.xlim()[0]], [plt.xlim()[1], plt.xlim()[1]], linestyle='--', color="grey")
#         plt.xlabel("Actual log10(C)")
#         plt.ylabel("Predicted log10(C)")
#         plt.title(modification)
        
        plot_data_df = pd.DataFrame({
            'Actual_log10(C)': y_test_reg_split,
            'Predicted_log10(C)': svr_pred
        })

        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_svr{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        plot_data_df.to_csv(save_file_path, index=False)

        print(f"SVR plot data saved to {save_file_path}")

        
        # Step 3: RF calculation
        # Create an rf
        rf = RandomForestRegressor(n_estimators=100, criterion="squared_error", max_depth=None, n_jobs=-1, verbose=0)
        rf.fit(X_train_split, y_train_reg_split_label)
        rf_pred = rf.predict(X_test_split)

        # Saving performances and pred vs real curve
        MAE = mean_absolute_error(y_test_reg_split_label, rf_pred)
        RMSE = np.sqrt(mean_squared_error(y_test_reg_split_label, rf_pred))
        R_squared = r2_score(y_test_reg_split_label, rf_pred)

        print("MAE:", MAE)
        print("RMSE:", RMSE)
        print("R^2:", R_squared)
        
        # Create a DataFrame with these metrics
        performance_metrics_df = pd.DataFrame({
            'MAE': [MAE],
            'RMSE': [RMSE],
            'R^2': [R_squared]
        })
        
        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_performance_rf{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        performance_metrics_df.to_csv(save_file_path, index=False)
        
        # Actual and predicted values
#         plt.figure(figsize=(6, 6))
#         plt.scatter(y_test_reg_split, rf_pred, edgecolors="grey", linewidths=0.2, alpha = 0.3)
#         plt.axline([plt.xlim()[0], plt.xlim()[0]], [plt.xlim()[1], plt.xlim()[1]], linestyle='--', color="grey")
#         plt.xlabel("Actual log10(C)")
#         plt.ylabel("Predicted log10(C)")
#         plt.title(modification)

        plot_data_df = pd.DataFrame({
            'Actual_log10(C)': y_test_reg_split,
            'Predicted_log10(C)': rf_pred
        })

        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_rf{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        plot_data_df.to_csv(save_file_path, index=False)

        print(f"RF plot data saved to {save_file_path}")


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\1142750743.py:10: DtypeWarning: Columns (4197) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


NaN values found in the DataFrame. Imputing with median values...


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\1142750743.py:33: DtypeWarning: Columns (4197) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(tfile_path)


NaN values found in the DataFrame. Imputing with median values...
MAE: 2.103009271684485
RMSE: 3.1399892603184694
R^2: 0.6434468258754898
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\3x3_1_FigureData\Bare_Cysteamine_Cysteine_svr1.csv
MAE: 0.2899642037838865
RMSE: 0.5040048229979135
R^2: 0.9908137689089521
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\3x3_1_FigureData\Bare_Cysteamine_Cysteine_rf1.csv


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\1142750743.py:10: DtypeWarning: Columns (4197) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


...

NaN values found in the DataFrame. Imputing with median values...
MAE: 2.0327553782058967
RMSE: 3.0838644873925047
R^2: 0.6288526631190338
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\3x3_10_FigureData\Bare_Cysteamine_MCH_svr10.csv
MAE: 1.1479720195861767
RMSE: 2.7693195142143514
R^2: 0.7007033133762302
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\3x3_10_FigureData\Bare_Cysteamine_MCH_rf10.csv


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\1142750743.py:10: DtypeWarning: Columns (4197) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


NaN values found in the DataFrame. Imputing with median values...


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\1142750743.py:33: DtypeWarning: Columns (4197) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(tfile_path)


NaN values found in the DataFrame. Imputing with median values...
MAE: 2.1613510842468027
RMSE: 2.9894893850407915
R^2: 0.651221417422801
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\3x3_10_FigureData\Bare_Cysteine_MCH_svr10.csv
MAE: 0.43400261945735424
RMSE: 0.6736427943125962
R^2: 0.9822901405570359
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\3x3_10_FigureData\Bare_Cysteine_MCH_rf10.csv
NaN values found in the DataFrame. Imputing with median values...
NaN values found in the DataFrame. Imputing with median values...
MAE: 0.7317920231140762
RMSE: 0.9591374530322574
R^2: 0.9640980963638202
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\3x3_10_FigureData\Cysteamine_Cysteine_MCH_svr10.csv
MAE: 0.45561687871054596
RMSE: 0.6984061222084024
R^2: 0.9809641684742179
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\

### 4x4 superspectra

In [18]:
# set the number of total iterations (groups)
iteration_number = 10
START = 1 # Included

In [19]:
for i in range(START, START+iteration_number):
    modification_combination_list = ['Bare_Cysteamine_Cysteine_MCH']
    
    for modification in modification_combination_list:
        # Step 1: reformat the train and test sets
        # train
        file_path = f"{WORKING_PATH}\\Superspectra20250705\\4x4_{i}\\{modification}_train{i}.csv"

        # Read the CSV file into a DataFrame
        train = pd.read_csv(file_path)
        train = preprocess_data(train)

#         # Display the first few rows of the data to verify
#         print(train.head())

        # Select training data with a specific label
        selected_label = train['label'].iloc[0] # always choose the first label. This label has no use, but just to ensure that the code can run.
        selected_data_train = train[train['label'] == selected_label] # split training data
        # selected_data_train

        # Get the indices of the selected data
        selected_indices_train = selected_data_train.index
        # selected_indices_train

        # Get new X_train and y_train_reg
        X_train_split = selected_data_train.values[:,:-2]
        y_train_reg_split = np.log10(selected_data_train.values[:,-1].astype(float)+1e-20) # we only need the regression part.

        # test
        tfile_path = f"{WORKING_PATH}\\Superspectra20250705\\4x4_{i}\\{modification}_test{i}.csv"  # Replace with the path to your CSV file

        # Read the CSV file into a DataFrame
        test = pd.read_csv(tfile_path)
        test = preprocess_data(test)

#         # Display the first few rows of the data to verify
#         print(test.head())

        # Similarly, select testing data with the specific label
        selected_data_test = test[test['label'] == selected_label] # split testing data
        # selected_data_test

        # Get the indices of the selected data
        selected_indices_test = selected_data_test.index
        # selected_indices_test

        # Get new X_test and y_test_reg
        X_test_split = selected_data_test.values[:,:-2]
        y_test_reg_split = np.log10(selected_data_test.values[:,-1].astype(float)+1e-20) # we only need the regression part.
     
    
        # Step 2: SVR calculation
        # Create a svr
        svr = svm.SVR(kernel = 'rbf', C = 1000, epsilon = 0.1)
        # sklearn.svr only accept label vectors as input, not one-hot vectors.
        y_train_reg_split_label = y_train_reg_split
        with parallel_backend('threading', n_jobs=-1):
            # Fit the SVR model
            svr.fit(X_train_split, y_train_reg_split_label)
        svr_pred = svr.predict(X_test_split)
        
        # Comparing results
        y_test_reg_split_label = y_test_reg_split

        
        # Saving performances and pred vs real curve
        MAE = mean_absolute_error(y_test_reg_split_label, svr_pred)
        RMSE = np.sqrt(mean_squared_error(y_test_reg_split_label, svr_pred))
        R_squared = r2_score(y_test_reg_split_label, svr_pred)

        print("MAE:", MAE)
        print("RMSE:", RMSE)
        print("R^2:", R_squared)

        save_directory = f"{WORKING_PATH}\\Superspectra20250705\\Results\\4x4_{i}_FigureData"
        os.makedirs(save_directory, exist_ok=True)
        
        # Create a DataFrame with these metrics
        performance_metrics_df = pd.DataFrame({
            'MAE': [MAE],
            'RMSE': [RMSE],
            'R^2': [R_squared]
        })
        
        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_performance_svr{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        performance_metrics_df.to_csv(save_file_path, index=False)
        
        # Actual and predicted values
#         plt.figure(figsize=(6, 6))
#         plt.scatter(y_test_reg_split, svr_pred, edgecolors="grey", linewidths=0.2, alpha = 0.3)
#         plt.axline([plt.xlim()[0], plt.xlim()[0]], [plt.xlim()[1], plt.xlim()[1]], linestyle='--', color="grey")
#         plt.xlabel("Actual log10(C)")
#         plt.ylabel("Predicted log10(C)")
#         plt.title(modification)
        
        plot_data_df = pd.DataFrame({
            'Actual_log10(C)': y_test_reg_split,
            'Predicted_log10(C)': svr_pred
        })

        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_svr{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        plot_data_df.to_csv(save_file_path, index=False)

        print(f"SVR plot data saved to {save_file_path}")

        
        # Step 3: RF calculation
        # Create an rf
        rf = RandomForestRegressor(n_estimators=100, criterion="squared_error", max_depth=None, n_jobs=-1, verbose=0)
        rf.fit(X_train_split, y_train_reg_split_label)
        rf_pred = rf.predict(X_test_split)

        # Saving performances and pred vs real curve
        MAE = mean_absolute_error(y_test_reg_split_label, rf_pred)
        RMSE = np.sqrt(mean_squared_error(y_test_reg_split_label, rf_pred))
        R_squared = r2_score(y_test_reg_split_label, rf_pred)

        print("MAE:", MAE)
        print("RMSE:", RMSE)
        print("R^2:", R_squared)

        # Create a DataFrame with these metrics
        performance_metrics_df = pd.DataFrame({
            'MAE': [MAE],
            'RMSE': [RMSE],
            'R^2': [R_squared]
        })
        
        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_performance_rf{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        performance_metrics_df.to_csv(save_file_path, index=False)
        
        # Actual and predicted values
#         plt.figure(figsize=(6, 6))
#         plt.scatter(y_test_reg_split, rf_pred, edgecolors="grey", linewidths=0.2, alpha = 0.3)
#         plt.axline([plt.xlim()[0], plt.xlim()[0]], [plt.xlim()[1], plt.xlim()[1]], linestyle='--', color="grey")
#         plt.xlabel("Actual log10(C)")
#         plt.ylabel("Predicted log10(C)")
#         plt.title(modification)

        plot_data_df = pd.DataFrame({
            'Actual_log10(C)': y_test_reg_split,
            'Predicted_log10(C)': rf_pred
        })

        # Construct the new file path with the same filename in the new directory
        save_file_path = os.path.join(save_directory, f"{modification}_rf{i}.csv")
        # Save the DataFrame to a CSV file in the new location with the same name
        plot_data_df.to_csv(save_file_path, index=False)

        print(f"RF plot data saved to {save_file_path}")


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\3668949453.py:10: DtypeWarning: Columns (5596) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


NaN values found in the DataFrame. Imputing with median values...


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\3668949453.py:33: DtypeWarning: Columns (5596) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(tfile_path)


NaN values found in the DataFrame. Imputing with median values...
MAE: 1.6115564998852203
RMSE: 2.2061109746861254
R^2: 0.8100626006753281
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\4x4_1_FigureData\Bare_Cysteamine_Cysteine_MCH_svr1.csv
MAE: 0.5932159643638169
RMSE: 0.8792681360253783
R^2: 0.9698283926696337
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\4x4_1_FigureData\Bare_Cysteamine_Cysteine_MCH_rf1.csv


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\3668949453.py:10: DtypeWarning: Columns (5596) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


...

NaN values found in the DataFrame. Imputing with median values...
MAE: 1.755025881522555
RMSE: 2.4763544450614985
R^2: 0.760678680141083
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\4x4_9_FigureData\Bare_Cysteamine_Cysteine_MCH_svr9.csv
MAE: 0.5125361263780763
RMSE: 0.7625605918341166
R^2: 0.9773063426873821
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\4x4_9_FigureData\Bare_Cysteamine_Cysteine_MCH_rf9.csv


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\3668949453.py:10: DtypeWarning: Columns (5596) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(file_path)


NaN values found in the DataFrame. Imputing with median values...


C:\Users\zhao-\AppData\Local\Temp\ipykernel_11060\3668949453.py:33: DtypeWarning: Columns (5596) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(tfile_path)


NaN values found in the DataFrame. Imputing with median values...
MAE: 2.028620025788081
RMSE: 2.731548009602658
R^2: 0.7088120136598731
SVR plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\4x4_10_FigureData\Bare_Cysteamine_Cysteine_MCH_svr10.csv
MAE: 0.44785629717103836
RMSE: 0.6613502698593456
R^2: 0.9829305767500988
RF plot data saved to C:\Users\zhao-\Jiaheng\BSA protein\data\Superspectra20231107\Results\4x4_10_FigureData\Bare_Cysteamine_Cysteine_MCH_rf10.csv
